# Introduction 
The aim of this notebook is to figure out how to draw a map with 'plotly' step by step.

## Content
1. [Custom Map with json](#1)
     - [Animation of Turkey Population yearly](#1)
     - [Turkey 2000-2019 internal migration rate](#2)
1. [World Map with express plotly](#3)
     - [Netflix data set, average movie duration](#3)




In [ ]:
# import libraries 
# data cleaning
import pandas as pd
import numpy as np

#data visualization
import matplotlib as mpl
import plotly.graph_objects as go
import plotly.express as px

import plotly.offline as pyo
import plotly.graph_objs as go
#visualazation libraries
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()

#to close warnings
import warnings
warnings.filterwarnings("ignore")

Getting json file with coordination of the place. The important thing here, you need an id variable in json data set to merge with the choropleth map. In this data set each id represents that each city. Additionally, I made this data set name is 'cities', I will use this file below to draw the map.

In [ ]:
#this library necessary for map
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/cihadturhan/tr-geojson/master/geo/tr-cities-utf8.json') as response:
    cities = json.load(response)
#cities["features"][0]

In [ ]:
#load data sets
df = pd.read_csv('../input/trpopulation/TRNufus.csv')
df_im = pd.read_csv('../input/migratetr2/MigRate.csv')


In [ ]:
#column name configuration, actually this is not necessary for this short visualization but it helps me for standardization
df.rename(columns = {'Number':'id','City':'city','Pop':'pop', 'Year':'year'},inplace =True)

In [ ]:
#this step needed for json data and csv data merge, these two data sets merge with id
df.set_index('Id', inplace=True)

# Population of Turkey

In the below map, I want to show that just choropleth does not provide make zoom, unfortuanetly. Additionally,  because Istanbul has the majortiy of population in Turkey over the years, the color scale does not change prominently. Because I made each city's id index, I used 'df.index', and my json file name is 'cities', so I used for geojson. For colour palette, you can choose any of [them](https://plotly.com/python/builtin-colorscales/).

In [ ]:


fig = px.choropleth(df, geojson=cities, locations=df.index, color="pop", 
                    hover_name="city", animation_frame=df["year"],color_continuous_scale="earth",
                    
                         
                           labels={'pop':'population'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In this map, I used "choropleth_mapbox" , and I could use zoom for Turkey. And, I used "np.log10" for population normalization to minimize  the effect of Istanbul population. So, slightly we can see change of colors over the years.

In [ ]:
fig = px.choropleth_mapbox(df, geojson=cities, locations=df.index, color=np.log10(df["pop"]),hover_name="city", animation_frame=df["year"],
                           color_continuous_scale="Viridis",
                           
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 38.963745, "lon": 35.243322},
                           opacity=0.7,
                           labels={'color':'population','Id': 'city','population':'pop'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.choropleth_mapbox(df, geojson=cities, locations=df.index, color=np.log10(df["pop"]),hover_name="city", animation_frame=df["year"],
                           color_continuous_scale='twilight',
                           
                           mapbox_style="carto-positron",
                           zoom=4, center = {"lat": 38.963745, "lon": 35.243322},
                           opacity=0.7,
                           labels={'color':'population','Id': 'city'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Turkey 2000-2019 internal migration rate

to clearly see change of the years, I used internal migration rate. And, As you can see below, it can be seen clearly, generally internal migration rate has been decreasing over the years for cities of Turkey.

In [ ]:
df_im.rename(columns = {'Id':'id','City':'city','Rate':'rate', 'Year':'year'},inplace =True)

In [ ]:
df_im.set_index('id', inplace=True)

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(df_im, geojson=cities, locations=df_im.index, color='rate',hover_name="city", animation_frame=df_im["year"],
                           color_continuous_scale='twilight',
                           
                           mapbox_style="carto-positron",
                           zoom=5, center = {"lat": 38.963745, "lon": 35.243322},
                           opacity=0.7,
                           labels={'color':'rate','id': 'city'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


# WORLD MAP

For drawing a world map, you don't need to json file, country names or countries' ISO codes enough to draw it. For example, I chose the Netflix data set, and I used country names to draw it. The only numerical variable is the duration and it is given as an object, so firstly I made little bit data wrangling, I split number from text and convert to integer than I made a map with default options. Additionally, I also selected the first country from the country column, if I didn't do this step, it still could draw a map but hover information of countries looked a little bit confusing.

In [ ]:
df_movie=pd.read_csv("../input/netflix-shows/netflix_titles.csv")#reading file
df_movie.sample(10)#look at into file

In [ ]:
df0 = df_movie[df_movie['type'] == 'Movie'] #select movies
name = df0['duration']
df0['duration'] = [i.split(" ")[0].strip() for i in name]#split number and text
df0['duration'] = df0['duration'].astype(str).astype(float)

name2 = df0['country'].astype(str)
df0['Country'] = [i.split(",")[0].strip() for i in name2]#split number and text


df0 = df0.groupby(['Country',]).agg( { 'duration':'mean', }) #getting average duration each country or countries
df0.reset_index(inplace = True)


In [ ]:

fig = px.choropleth(df0, locations="Country", # used plotly express choropleth for animation plot
                    color="duration", 
                    locationmode='country names',
                    hover_name="Country",
                    hover_data=['duration'],
                    title = 'Average Duration of Movies',
                   color_continuous_scale='OrRd')

# adjusting size of map, legend place, and background colour
fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    legend=dict(
        orientation="v",
        yanchor="auto",
        y=1.02,
        xanchor="right",
        x=1
))

fig.show()
# reference: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html

## Creating Discrete Legend 
In this section, I wanted to draw a more clear map, so I created a new Duration column and split durations for six categories. Because most countries have an average duration of around 80 and 120 min, I split according to this gap. And, I also added an annotation for an example. I used 'category_orders' with the same sequence and I coloured them using a colour palette from [this useful website](https://colorbrewer2.org/#type=sequential&scheme=YlGn&n=6).





In [ ]:
df_map = df0.copy() # do not lose 'df', I made copy of it
df_map['Duration'] = ['d<=80' if x<=80 else '80<d<=90' if (80)<x<=(90) else '90<d<=100' if (90)<x<=100 else '100<d<=110' if (100)<x<=110 else '110<d<=120' if 110<x<=120 else '120<d<=200' if 120<x<=200  else 'None' for x in df_map['duration']]
# categorized each of temperature changes

In [ ]:
fig = px.choropleth(df_map, locations="Country", # used plotly express choropleth for animation plot
                    color="Duration", 
                    locationmode='country names',
                    hover_name="Country",
                    hover_data=['Duration','duration'],
                    
                    labels={'duration':'Duration', 'Duration':'Duration category'},
                    category_orders={'Duration':['d<=80','80<d<=90','90<d<=100','100<d<=110','110<d<=120','120<d<=200','None']},
                    color_discrete_map={'d<=80':"#ffffcc",'80<d<=90':"#d9f0a3",'90<d<=100':"#addd8e",'100<d<=110':"#78c679",'110<d<=120':"#31a354",'120<d<=200': "#006837",'None':"#252525"},
                    title = 'Average Duration of Movies',
                   color_continuous_scale='OrRd')

# adjusting size of map, legend place, and background colour
fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    template='seaborn',
    paper_bgcolor="white",
    legend=dict(
        orientation="v",
        yanchor="auto",
        y=1.02,
        xanchor="right",
        x=1
))

fig.add_annotation(
        x=0.3,
        y=0.68,
        text="USA",
        showarrow=True,
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#636363",
        ax=20,
        ay=-30,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )


fig.show()
# reference1: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html
#reference2: https://plotly.com/python/text-and-annotations/#styling-and-coloring-annotations

# If you find it useful,  please upvote! Thanks!